In [3]:
%pip install pillow opencv-python pydub

Note: you may need to restart the kernel to use updated packages.


In [4]:
import zlib

def compress_deflate(data):
    return zlib.compress(data)

def decompress_deflate(data):
    return zlib.decompress(data)


In [5]:
import lzma

def compress_lzma(data):
    return lzma.compress(data)

def decompress_lzma(data):
    return lzma.decompress(data)


In [6]:
import time

def measure_compression(func, data):
    start_time = time.time()
    compressed_data = func(data)
    end_time = time.time()
    duration = end_time - start_time
    size = len(compressed_data)
    return compressed_data, size, duration

def save_compressed_file(data, filename):
    with open(filename, 'wb') as file:
        file.write(data)


In [7]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox

def open_file():
    filepath = filedialog.askopenfilename()
    if filepath:
        file_label.config(text=f"Selected file: {filepath}")
        global selected_file_path
        selected_file_path = filepath

def compress_file():
    if not selected_file_path:
        messagebox.showerror("Error", "No file selected!")
        return
    
    try:
        with open(selected_file_path, 'rb') as file:
            data = file.read()
        
        global original_size, compressed_deflate, compressed_lzma, file_name, file_ext
        original_size = len(data)
        
        compressed_deflate, size_deflate, time_deflate = measure_compression(compress_deflate, data)
        compressed_lzma, size_lzma, time_lzma = measure_compression(compress_lzma, data)
        
        result = f"Original Size: {original_size} bytes\n\n"
        result += f"DEFLATE: Compressed Size = {size_deflate} bytes, Time = {time_deflate:.5f} seconds\n"
        result += f"LZMA: Compressed Size = {size_lzma} bytes, Time = {time_lzma:.5f} seconds"
        result_label.config(text=result)
        
        file_name, file_ext = os.path.splitext(os.path.basename(selected_file_path))
        
    except Exception as e:
        messagebox.showerror("Error", str(e))

def save_files():
    if not compressed_deflate or not compressed_lzma:
        messagebox.showerror("Error", "No compressed data to save!")
        return
    
    try:
        deflate_path = os.path.join('results', f'compressed_deflate_{file_name}{file_ext}')
        lzma_path = os.path.join('results', f'compressed_lzma_{file_name}{file_ext}')
        
        save_compressed_file(decompress_deflate(compressed_deflate), deflate_path)
        save_compressed_file(decompress_lzma(compressed_lzma), lzma_path)
        
        messagebox.showinfo("Success", f"Files saved as:\n{deflate_path}\n{lzma_path}")
    except Exception as e:
        messagebox.showerror("Error", str(e))

app = tk.Tk()
app.title("Compression Comparison")
app.geometry("600x400")

selected_file_path = ""
compressed_deflate = None
compressed_lzma = None
original_size = 0
file_name = ""
file_ext = ""

open_button = tk.Button(app, text="Open File", command=open_file)
open_button.pack(pady=10)

file_label = tk.Label(app, text="No file selected")
file_label.pack(pady=10)

compress_button = tk.Button(app, text="Compress", command=compress_file)
compress_button.pack(pady=10)

result_label = tk.Label(app, text="", justify="left")
result_label.pack(pady=10)

save_button = tk.Button(app, text="Save Compressed Files", command=save_files)
save_button.pack(pady=10)

app.mainloop()